## 講座1_シンプルバージョンをベースにする

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
pwd

'/content'

In [3]:
!nvidia-smi

Wed Jul 21 10:28:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os

import pandas as pd
import numpy as np
from glob import  glob

import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
input_dir = '/content/drive/MyDrive/Colab Notebooks/atmacup/atmacup11/data/inputs/'
photo_dir = os.path.join(input_dir, 'photos')
photo_pathes = glob(os.path.join(photo_dir, "*.jpg"))
output_dir = '/content/drive/MyDrive/Colab Notebooks/atmacup/atmacup11/data/outputs/'
model_dir = '/content/drive/MyDrive/Colab Notebooks/atmacup/atmacup11/data/model/'

os.makedirs(output_dir, exist_ok=True)

train_df = pd.read_csv(os.path.join(input_dir, 'train.csv'))
test_df = pd.read_csv(os.path.join(input_dir, 'test.csv'))

material_df = pd.read_csv(os.path.join(input_dir, 'materials.csv'))
technique_df = pd.read_csv(os.path.join(input_dir, 'techniques.csv'))

In [26]:
import re
from requests import get

#!pip install ipynb-path
#import ipynb_path
# dimension of the embeddings
num_ftrs = 512
# dimension of the output of the prediction and projection heads
out_dim = proj_hidden_dim = 512
# the prediction head uses a bottleneck architecture
#pred_hidden_dim = 128
# use 2 layers in the projection head
num_mlp_layers = 2

class Config:
    N_FOLDS = 5
    N_EPOCHS = 50
    #NB_NAME = ''.join(re.findall('.*/(.*).ipynb', ipynb_path.get()))
    nb_name = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
    NB_NAME = re.findall('(.*).ipynb', nb_name)[0]

## seedの固定

In [7]:
import torch
import random

def seed_torch(seed=1993):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

### 画像データの読み込み

In [8]:
from PIL import Image

def to_img_path(object_id):
    return os.path.join(photo_dir, f'{object_id}.jpg')

def read_image(object_id):
    return Image.open(to_img_path(object_id))

In [9]:
!pip uninstall -y scikit-learn
!pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn
!pip install lightly
# ランタイム再起動したらimportできるようになりまっせ

Found existing installation: scikit-learn 0.22.2.post1
Uninstalling scikit-learn-0.22.2.post1:
  Successfully uninstalled scikit-learn-0.22.2.post1
Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/scipy-wheels-nightly/simple
     |████████████████████████████████| 22.9 MB 87.5 MB/s 
     |████████████████████████████████| 240 kB 15.2 MB/s 
     |████████████████████████████████| 813 kB 23.5 MB/s 
     |████████████████████████████████| 76 kB 5.8 MB/s 
     |████████████████████████████████| 144 kB 59.8 MB/s 
     |████████████████████████████████| 74 kB 3.6 MB/s 
     |████████████████████████████████| 112 kB 71.6 MB/s 
     |████████████████████████████████| 636 kB 61.8 MB/s 
     |████████████████████████████████| 234 kB 34.7 MB/s 
     |████████████████████████████████| 118 kB 69.8 MB/s 
     |████████████████████████████████| 829 kB 72.8 MB/s 
     |████████████████████████████████| 10.6 MB 58.7 MB/s 
     |████████████████████████████████| 1.3 MB 53.6 MB/s 
 

In [12]:
import torch
from torch import nn
from torch.optim import Adam
from torch.optim.optimizer import Optimizer
from torch.utils import data

# torchvision
from torchvision import transforms as T
from torchvision.models import resnet18

# scikit-learn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedGroupKFold

import lightly

In [13]:
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]

class AtmaDataset(data.Dataset):
    """atmaCup用にデータ読み込み等を行なうデータ・セット"""
    object_path_key = "object_path"
    label_key = "target"

    @property
    def meta_keys(self):
        retval = [self.object_path_key]

        if self.is_train:
            retval += [self.label_key]

        return retval

    def __init__(self, meta_df: pd.DataFrame, is_train=True):
        """
        args:
            meta_df: 
                画像へのパスと label 情報が含まれている dataframe
                必ず object_path に画像へのパス, target に正解ラベルが入っている必要があります
            
            is_train:
                True のとき学習用のデータ拡張を適用します.
                False の時は単に size にリサイズを行います
        """

        self.is_train = is_train
        self.meta_df = meta_df.reset_index(drop=True)
        self.index_to_data = self.meta_df.to_dict(orient="index")

        size = (224, 224)

        additional_items = (
            [T.Resize(size)]
            if not is_train
            else [
                T.RandomGrayscale(p=0.2),
                T.RandomVerticalFlip(),
                T.RandomHorizontalFlip(),
                T.RandomResizedCrop(size),
            ]
        )

        self.transformer = T.Compose(
            [*additional_items, T.ToTensor(), T.Normalize(mean=IMG_MEAN, std=IMG_STD)]
        )

    def __getitem__(self, index):
        data = self.index_to_data[index]

        obj_path, label = data.get(self.object_path_key), data.get(self.label_key, -1)
        img = Image.open(obj_path)
        img = self.transformer(img)
        return img, label

    def __len__(self):
        return len(self.meta_df)

In [14]:
assert torch.cuda.is_available()

DEVICE = torch.device("cuda")

## Train / Validation Phase

In [15]:
def train(
    model: nn.Module,
    optimizer: Optimizer,
    train_loader: data.DataLoader
) -> pd.Series:
    
    # train にすることで model 内の学習時にのみ有効な機構が有効になります (Dropouts Layers、BatchNorm Layers
    model.train()
    
    criterion = nn.MSELoss()
    
    for i, (x_i, y_i) in enumerate(train_loader):
        x_i = x_i.to(DEVICE)
        y_i = y_i.to(DEVICE).reshape(-1,1).float()
        
        output = model(x_i)
        loss = criterion(output, y_i)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

def predict(model: nn.Module, loader: data.DataLoader) -> np.ndarray:
    # train とは逆で model 内の学習時にのみ有効な機構がオフになります (Dropouts Layers、BatchNorm Layers...)
    model.eval()
    predicts = []
    
    for x_i, y_i in loader:
        
        # 明示的に勾配を計算しないように指定することができます. 
        # この関数ではモデルの更新はせずに単に出力だけを使いますので勾配は不要です.
        with torch.no_grad():
            output = model(x_i.to(DEVICE))
            
        predicts.extend(output.data.cpu().numpy())
        
    pred = np.array(predicts).reshape(-1)
    return pred

def calculate_metrics(y_true, y_pred) -> dict:
    """正解ラベルと予測ラベルから指標を計算する"""    
    return {
        'rmse': mean_squared_error(y_true, y_pred) ** .5
    }

def valid(
    model: nn.Module, 
    y_valid: np.ndarray, 
    valid_loader: data.DataLoader
) -> pd.Series:
    """検証フェーズ
    与えられたモデル・データローダを使って検証フェーズを実行。スコアの dict と予測した値を返す
    """
    
    pred = predict(model, valid_loader)
    print(f'y_valid={y_valid.shape} pred={pred.shape}')
    score = calculate_metrics(y_valid, pred)
    return score, pred
        

## Run Fold

1. train / valid の loader 作成
2. 以下を epoch 数だけ繰り返す
    1. 学習用データで学習 
    2. 検証用データで検証スコアの算出

In [16]:
def run_fold(
    model: nn.Module, 
    train_df: pd.DataFrame, 
    valid_df: pd.DataFrame, 
    y_valid: np.ndarray, 
    n_epochs=30) -> np.ndarray:
    """
    train / valid に分割されたデータで学習と同時に検証を行なう
    """
    
    # 0: 
    #   : 前準備. dataframe から data loader を作成
    train_dataset = AtmaDataset(meta_df=train_df)
    train_loader = data.DataLoader(
        train_dataset, batch_size=256, shuffle=True, drop_last=True, num_workers=2
    )
    
    #   : 検証用の方は is_train=False にしてデータ拡張オフにする
    valid_dataset = AtmaDataset(meta_df=valid_df, is_train=False)
    valid_loader = data.DataLoader(valid_dataset, batch_size=256, num_workers=2)
    
    # optimizer の定義
    optimizer = Adam(model.parameters(), lr=1e-3)
    
    for epoch in range(1, n_epochs + 1):
        print(f'start {epoch}')
        
        # 1: 学習用データで学習を実行。学習時のロスを取得
        train(model, optimizer, train_loader)
        
        # 2: 検証データでのスコアを計算
        score_valid, y_valid_pred = valid(model=model, valid_loader=valid_loader, y_valid=y_valid)
        
        print(score_valid)

### その他

モデル作成などの関数定義

In [17]:
#import timm

# def create_model():
#     model = timm.create_model('efficientnet_b3', pretrained=False)
#     model.fc = nn.Linear(in_features=512, out_features=1, bias=True)
#     return model


def create_model():
    resnet = resnet18(pretrained=False)
    backbone = nn.Sequential(*list(resnet.children())[:-1])
    model = lightly.models.SimSiam(
        backbone,
        num_ftrs=num_ftrs,
    #     proj_hidden_dim=pred_hidden_dim,
    #     pred_hidden_dim=pred_hidden_dim,
    #     out_dim=out_dim,
        num_mlp_layers=num_mlp_layers
    )
    model.load_state_dict(torch.load(os.path.join(model_dir,'simsiam_res18_256.pth')))
    model = model.backbone
    model.add_module('flatten', nn.Flatten())
    model.add_module('fc', nn.Linear(in_features=512, out_features=1, bias=True))
    return model

def create_metadata(input_df):
    out_df = input_df[['object_id']].copy()
    out_df['object_path'] = input_df['object_id'].map(to_img_path)
    
    if "target" in input_df:
        out_df["target"] = input_df["target"]
        
    return out_df

from tqdm import tqdm 
def run_test_predict(model, n_tta=0):
    test_meta_df = create_metadata(test_df)
    
    # n_tta > 0 の時だけデータ拡張を on にする (is_train = True)
    is_tta_mode = n_tta > 0
    test_dataset = AtmaDataset(meta_df=test_meta_df, is_train=is_tta_mode)
    test_loader = data.DataLoader(dataset=test_dataset, batch_size=256, drop_last=False, num_workers=4)
    
    predictions = []
    n_times = 1 if not is_tta_mode else n_tta
    print(f"run #{n_times} times / tta={is_tta_mode}")
    for _ in tqdm(range(n_times)):
        y_pred = predict(model, loader=test_loader)
        predictions.append(y_pred)
    
    return np.array(predictions).mean(axis=0)

In [18]:
# from torchsummary import summary
# #model = resnet18(pretrained=False)
# resnet = resnet18(pretrained=False)
# backbone = nn.Sequential(*list(resnet.children())[:-1])
# model = lightly.models.SimSiam(
#     backbone,
#     num_ftrs=num_ftrs,
# #     proj_hidden_dim=pred_hidden_dim,
# #     pred_hidden_dim=pred_hidden_dim,
# #     out_dim=out_dim,
#     num_mlp_layers=num_mlp_layers
# )
# model.load_state_dict(torch.load(os.path.join(model_dir,'simsiam_res18_256.pth')))
# model = model.backbone
# model.add_module('flatten', nn.Flatten())
# model.add_module('fc', nn.Linear(in_features=512, out_features=1, bias=True))
# DEVICE = torch.device("cuda")
# model.to(DEVICE)
# summary(model, (3,224,224))

In [19]:
train_meta_df = create_metadata(train_df)

test_predictions = []

fold = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=510)
cv = list(fold.split(X= train_df, y=train_df["target"], groups=train_df["art_series_id"]))[:Config.N_FOLDS]

for i, (idx_tr, idx_valid) in enumerate(cv):
    print(f'==== start cv {i} ====')
    model = create_model()
    model.to(DEVICE)
    
    # 1. Fold の学習
    run_fold(
        model=model, 
        train_df=train_meta_df.iloc[idx_tr], 
        valid_df=train_meta_df.iloc[idx_valid], 
        y_valid=train_meta_df['target'].values[idx_valid],
        n_epochs=Config.N_EPOCHS
    )
    
    # 2. モデルで予測 (本当はローカルに保存した重みを読みだすなどするほうがあとで振り返りやすいが簡易にそのまま予測する)
    y_pred_i = run_test_predict(model, 10)
    test_predictions.append(y_pred_i)
    del model

==== start cv 0 ====
start 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


y_valid=(778,) pred=(778,)
{'rmse': 11.831408082529453}
start 2
y_valid=(778,) pred=(778,)
{'rmse': 2.163181470117352}
start 3
y_valid=(778,) pred=(778,)
{'rmse': 1.0239011665683344}
start 4
y_valid=(778,) pred=(778,)
{'rmse': 0.9010882810970146}
start 5
y_valid=(778,) pred=(778,)
{'rmse': 0.87857819854826}
start 6
y_valid=(778,) pred=(778,)
{'rmse': 0.9547556906643763}
start 7
y_valid=(778,) pred=(778,)
{'rmse': 1.0497142644872233}
start 8
y_valid=(778,) pred=(778,)
{'rmse': 0.9096545794039902}
start 9
y_valid=(778,) pred=(778,)
{'rmse': 0.9093402199471574}
start 10
y_valid=(778,) pred=(778,)
{'rmse': 0.9078389121388455}
start 11
y_valid=(778,) pred=(778,)
{'rmse': 0.8810302939544666}
start 12
y_valid=(778,) pred=(778,)
{'rmse': 0.9931628923367427}
start 13
y_valid=(778,) pred=(778,)
{'rmse': 1.0010050658390799}
start 14
y_valid=(778,) pred=(778,)
{'rmse': 0.9393348715118731}
start 15
y_valid=(778,) pred=(778,)
{'rmse': 1.1363298842819798}
start 16
y_valid=(778,) pred=(778,)
{'rmse': 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/10 [00:00<?, ?it/s]

y_valid=(778,) pred=(778,)
{'rmse': 1.0982956356044473}
run #10 times / tta=True


...

100%|██████████| 10/10 [03:03<00:00, 18.35s/it]


==== start cv 4 ====
start 1
y_valid=(796,) pred=(796,)
{'rmse': 2.294087176325201}
start 2
y_valid=(796,) pred=(796,)
{'rmse': 1.3196197777435745}
start 3
y_valid=(796,) pred=(796,)
{'rmse': 1.119591046283418}
start 4
y_valid=(796,) pred=(796,)
{'rmse': 1.0354883147714553}
start 5
y_valid=(796,) pred=(796,)
{'rmse': 1.0916643302753948}
start 6
y_valid=(796,) pred=(796,)
{'rmse': 0.9238688124518144}
start 7
y_valid=(796,) pred=(796,)
{'rmse': 0.9906531503885089}
start 8
y_valid=(796,) pred=(796,)
{'rmse': 0.8994046014054201}
start 9
y_valid=(796,) pred=(796,)
{'rmse': 0.9329208048658342}
start 10
y_valid=(796,) pred=(796,)
{'rmse': 1.069233499869675}
start 11
y_valid=(796,) pred=(796,)
{'rmse': 0.9008372171587624}
start 12
y_valid=(796,) pred=(796,)
{'rmse': 0.9988313119189006}
start 13
y_valid=(796,) pred=(796,)
{'rmse': 0.9871897163876293}
start 14
y_valid=(796,) pred=(796,)
{'rmse': 1.3888709142714726}
start 15
y_valid=(796,) pred=(796,)
{'rmse': 1.3616418312538645}
start 16
y_valid

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/10 [00:00<?, ?it/s]

y_valid=(796,) pred=(796,)
{'rmse': 1.0194365375626995}
run #10 times / tta=True


100%|██████████| 10/10 [03:02<00:00, 18.30s/it]


In [28]:
# すべての予測の平均値を使う
pred_mean = np.array(test_predictions).mean(axis=0)

pd.DataFrame({
    "target": pred_mean
}).to_csv(os.path.join(output_dir, Config.NB_NAME + ".csv"), index=False)

In [ ]:
!pip install timm
import timm
timm.list_models

In [ ]:
timm.list_models(pretrained=False)

In [27]:
Config.NB_NAME

'nb012'